In [32]:
import requests
import math
import os

assert "/boat_catkin_ws/src/boat_environment_monitoring/vtr3_gui/vtr-ui/public" in os.getcwd(), "Incorrrect cwd!"

**Note:** Map URLs need to be re-generated after 24 hours

Generate them here: https://colab.research.google.com/drive/1Hd7Q7SOB73aozFVBFzz5IlEO6xbcOTXI#scrollTo=JfhwGD4rETBP

In [33]:
# RK McMillan Park
source = {
    "watermask_dir": "./WaterMasks/",
    "watermask_url": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/08b2c41f7db236cf8569e48dfcfc5fb2-fb54d558425254a5dcd67154b1e9c36b/tiles/",  # {z}/{x}/{y}
    "map_dir": "./Sentinel2Maps/",
    "map_url": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/92d8f695cbcd89c27d0b4dfed573c6f8-e96859f3c9da426296d1a13107c1f351/tiles/",  # {z}/{x}/{y}
    "top_left": (43.572089, -79.568696),  # (lat, lon)
    "bottom_right": (43.559421, -79.548831)  # (lat, lon)
}

In [41]:
# Nine Mile Lake Beach Park
source = {
    "watermask_dir": "./WaterMasks/",
    "watermask_url": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/5cb5614e7b559e33f081b54efb0616aa-206ce3a804023d7ce2dba8d026242674/tiles/",  # {z}/{x}/{y}
    "map_dir": "./Sentinel2Maps/",
    "map_url": "https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/9c8441fdff1ad803e11d4112e69a9ba2-3ab81914c2df12c4bce183399fc00e3f/tiles/",  # {z}/{x}/{y}
    "top_left": (45.503779, -80.072551),  # (lat, lon)
    "bottom_right": (45.432782, -80.030456)  # (lat, lon)
}

In [42]:
def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)


In [44]:
bounds = {
    "zoom": range(8, 19),
    "xs": [],
    "ys": []
}

for i in range(len(bounds["zoom"])):
    top_left = deg2num(source["top_left"][0], source["top_left"][1], bounds["zoom"][i])
    bottom_right = deg2num(source["bottom_right"][0], source["bottom_right"][1], bounds["zoom"][i])

    bounds["xs"].append(range(top_left[0], bottom_right[0] + 1))
    bounds["ys"].append(range(top_left[1], bottom_right[1] + 1))

print(bounds)

{'zoom': range(8, 19), 'xs': [range(71, 72), range(142, 143), range(284, 285), range(568, 569), range(1136, 1138), range(2273, 2275), range(4547, 4550), range(9095, 9100), range(18191, 18199), range(36382, 36398), range(72764, 72796)], 'ys': [range(91, 92), range(183, 184), range(366, 367), range(732, 734), range(1465, 1467), range(2930, 2933), range(5861, 5866), range(11722, 11732), range(23444, 23464), range(46889, 46927), range(93778, 93853)]}


In [45]:
def cache_tiles(source, bounds, watermask=False):
    type = "map"
    if watermask:
        type = "watermask"

    for i in range(len(bounds["zoom"])):
        z = bounds["zoom"][i]

        for x in bounds["xs"][i]:
            img_dir = source[type + "_dir"] + str(z) + "/" + str(x) + "/"
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            for y in bounds["ys"][i]:
                save_file = img_dir + str(y) + ".png"
                img_url = source[type + "_url"] + str(z) + "/" + str(x) + "/" + str(y)

                img_data = requests.get(img_url).content
                with open(save_file, "wb") as handler:
                    handler.write(img_data)


In [46]:
cache_tiles(source, bounds, False)
cache_tiles(source, bounds, True)

# Port Credit: 4m 18.6s
# Nine Mile Lake: 61m 21.8s